# 2023-08-31__STA_conntest

So, hi.
copying from prev nb:

So, for all 10 Ns;\
For 10 diff seeds;
for both exc, inh, and unconn;\
we conntest (maximum) 100 input spike trains.\
(Each test is comprised of calculating 101 STAs: one real and the rest with shuffled ISIs).

In [18]:
include("lib/Nto1.jl")

[IJulia init done]
using Revise … ✔
using Units … ✔
using Nto1AdEx … ✔
using ConnectionTests … ✔
using ConnTestEval … ✔
using StatsBase … ✔


In [3]:
duration = 10minutes
N = 6500

@time sim = Nto1AdEx.sim(N, duration);

  2.462874 seconds (1.14 M allocations: 987.502 MiB, 1.91% gc time, 30.60% compilation time)


(So even with native code caching in Julia 1.9, we still have 30% of time compilation here).

We decided we'd pick the 100 highest firing (exc and inh).\
And then generate some unconnecteds too..\
What's their firing rate? Maybe sample from the real ones :) hehe, sure.

In [21]:
exc = highest_firing(excitatory_inputs(sim))[1:100]
inh = highest_firing(inhibitory_inputs(sim))[1:100]
both = [exc..., inh...]
fr = spikerate.(both)
showsome(fr / Hz)

200-element Vector{Float64}:
   1: 97.5
   2: 58.9
   ⋮
  75: 16.8
   ⋮
 146: 12.5
   ⋮
 199: 8.87
 200: 8.86


In [7]:
include("lib\\plot.jl")

import PythonCall … ✔ (2.8 s)
import PythonPlot … ✔ (5.5 s)
using Sciplotlib … ✔ (0.8 s)
using PhDPlots … ✔
